<a href="https://colab.research.google.com/github/currencyfxjle/Napoleon_Execution-RiskManagement/blob/main/Napoleon_Execution%26RiskManagement_EA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# COST FUNCTION
# OPTIMIZATION VARIABLES
# FIXED PARAMETERS

**Cost Functions**
The cost functions will include:

- Net profit and loss (PnL)
- Maximum drawdown
- Daily and monthly profit/loss limits
- Optimization Variables

**The optimization variables for the strategy parameters will be:**

- tp_in_pips: Take profit distance in pips
- multiplier: Multiplier for lot size adjustment
- bb_period: Period for Bollinger Bands calculation
- bb_devfactor: Standard deviation factor for Bollinger Bands
- partial_profit_pct: Percentage of position to close at partial profit
- partial_profit_distance_pct: Percentage of tp_in_pips to set partial profit distance

**Fixed Parameters**
Fixed parameters include:

- initial_lot_size: 0.01
- max_lot_size: 7
- max_spread: 10
- _Point: 0

In [11]:
# STRATEGY EXECUTION FOR BUY & SELL

**STRATEGY DESCRIPTION ON BUY'S & SELL'S EXECUTION CRITERIA**

**Buys & Sells Execution Criteria**

The NapoleonEA strategy uses Bollinger Bands to determine buy and sell signals under two contexts: mean reversion and trend following.

- **Trend Following:**

Buy Signal (Trend Following):

A buy signal is generated if the price is below the upper Bollinger Band for two consecutive periods and the current closing price is above the current opening price.
This indicates a potential upward trend after the price has been trading below the upper band.


In [2]:
def check_buy_signal_trend(self):
    return (self.data.close[-2] < self.bbands.lines.top[-2] and
            self.data.close[-1] < self.bbands.lines.top[-1] and
            self.data.close[-1] > self.data.open[-1])

- **Sell Signal (Trend Following):**

A sell signal is generated if the price is above the lower Bollinger Band for two consecutive periods and the current closing price is below the current opening price.
This indicates a potential downward trend after the price has been trading above the lower band.


In [4]:
def check_sell_signal_trend(self):
    return (self.data.close[-2] > self.bbands.lines.bot[-2] and
            self.data.close[-1] > self.bbands.lines.bot[-1] and
            self.data.close[-1] < self.data.open[-1])

**Mean Reversion:**

- **Buy Signal (Mean Reversion):**

A buy signal is generated if the price crosses below the lower Bollinger Band in the previous period and then crosses back above it, or if the current price is below the lower band but higher than the low of the current period.
This suggests a potential reversal from oversold conditions.

In [5]:
def check_buy_signal_counter(self):
    return ((self.data.close[-2] > self.bbands.lines.bot[-2] and
             self.data.close[-1] < self.bbands.lines.bot[-1]) or
            (self.data.close[-1] < self.bbands.lines.bot[-1] and
             self.data.close[-1] > self.data.low[-1]))

**Sell Signal (Mean Reversion):**

A sell signal is generated if the price crosses above the upper Bollinger Band in the previous period and then crosses back below it, or if the current price is above the upper band but lower than the high of the current period.
This suggests a potential reversal from overbought conditions.

In [6]:
def check_sell_signal_counter(self):
    return ((self.data.close[-2] < self.bbands.lines.top[-2] and
             self.data.close[-1] > self.bbands.lines.top[-1]) or
            (self.data.close[-1] > self.bbands.lines.top[-1] and
             self.data.close[-1] < self.data.high[-1]))

**Execution:**

Once a buy or sell signal is detected, the strategy opens a market trade with the current lot size.
Take profit levels are set for partial and full exits based on the parameters.

In [7]:
def execute_trading_signals(self):
    if self.get_total_buy_orders_count() == 0 and (self.check_buy_signal_trend() or self.check_buy_signal_counter()):
        tp_price_full = self.data.close[0] + (self.params.tp_in_pips * self.params._Point * 10)
        tp_price_partial = self.data.close[0] + (self.params.tp_in_pips * self.params.partial_profit_distance_pct / 100 * self.params._Point * 10)
        self.order = self.open_market_trade(self.current_lot_size, tp_price_partial, tp_price_full)
    if self.get_total_sell_orders_count() == 0 and (self.check_sell_signal_trend() or self.check_sell_signal_counter()):
        tp_price_full = self.data.close[0] - (self.params.tp_in_pips * self.params._Point * 10)
        tp_price_partial = self.data.close[0] - (self.params.tp_in_pips * self.params.partial_profit_distance_pct / 100 * self.params._Point * 10)
        self.order = self.open_market_trade(-self.current_lot_size, tp_price_partial, tp_price_full)

In [12]:
# RISK MANAGEMENT

**RISK MANAGEMENT SECTION CRITERIA**

**Risk Management Approach**

The NapoleonEA strategy implements multiple layers of risk management, including drawdown control, daily profit/loss limits, and a Martingale approach for lot size adjustment.

- **Drawdown Management:**

The strategy tracks the highest account balance and calculates the drawdown from this peak.

If the drawdown exceeds a specified percentage (close_on_drawdown_percent), all trades are closed to prevent further losses.

In [8]:
def manage_drawdown(self):
    current_balance = self.broker.getvalue()
    if current_balance > self.highest_account_balance:
        self.highest_account_balance = current_balance
    drawdown = (self.highest_account_balance - current_balance) / self.highest_account_balance * 100
    if drawdown > self.params.close_on_drawdown_percent:
        self.log(f'Drawdown exceeded {self.params.close_on_drawdown_percent}%, closing all trades.')
        self.close_all_trades()

- **Daily and Monthly Limits:**

The strategy monitors daily losses and profits. If the daily loss exceeds a specified limit (daily_sl), all trades are closed.

Similarly, if the realized daily profit exceeds a specified limit (max_daily_profit), all trades are closed.


In [9]:
def check_daily_limits(self):
    daily_loss_limit = self.starting_balance * (self.params.daily_sl / 100)
    current_balance = self.broker.getvalue()
    daily_loss = self.starting_balance - current_balance
    if daily_loss > daily_loss_limit:
        self.log(f'Daily loss limit exceeded: {daily_loss} > {daily_loss_limit}, all trades closed.')
        self.close_all_trades()
    elif self.realized_daily_profit > self.params.max_daily_profit:
        self.log('Daily profit target achieved, all trades closed.')
        self.close_all_trades()

**Martingale Strategy:**

- **The Martingale approach adjusts the lot size based on winning and losing streaks:**

After a losing trade, the lot size is increased by a specified multiplier (multiplier) up to a maximum lot size (max_lot_size) to recover losses faster.

After a winning trade, the lot size is decreased to the initial lot size or reduced by the multiplier to lock in profits.

In [10]:
def adjust_lot_size(self):
    if self.winning_streak > 0:
        self.current_lot_size = max(self.params.initial_lot_size, self.current_lot_size / self.params.multiplier)
    elif self.losing_streak > 0:
        self.current_lot_size = min(self.params.max_lot_size, self.current_lot_size * self.params.multiplier)


**FULL CODE**

In [ ]:
!pip install backtrader pandas
import backtrader as bt
import pandas as pd
from datetime import datetime

In [ ]:
import backtrader as bt
import pandas as pd
import json

class NapoleonEA(bt.Strategy):
    params = (
        ('initial_lot_size', 0.01),
        ('tp_in_pips', 5),
        ('multiplier', 1.05),
        ('max_lot_size', 7),
        ('bb_period', 20),
        ('bb_devfactor', 1),
        ('max_spread', 10),
        ('_Point', 0.00001),
        ('max_daily_loss', 100),
        ('max_monthly_profit', 500),
        ('close_on_drawdown_percent', 5),
        ('max_daily_profit', 200),
        ('daily_sl', 1),
        ('partial_profit_pct', 50),
        ('partial_profit_distance_pct', 30),
    )

    def __init__(self):
        self.order = None
        self.open_trades = {}
        self.order_ref_to_trade_id = {}
        self.bbands = bt.indicators.BollingerBands(period=self.params.bb_period, devfactor=self.params.bb_devfactor)
        self.realized_daily_profit = 0
        self.realized_monthly_profit = 0
        self.last_check_date = self.data.datetime.datetime(0).date()
        self.highest_account_balance = self.broker.getvalue()
        self.lot_multiplier = self.params.multiplier
        self.equity_curve = []
        self.max_drawdown = 0
        self.starting_balance = self.broker.getvalue()
        self.winning_streak = 0
        self.losing_streak = 0
        self.current_lot_size = self.params.initial_lot_size

    def notify_order(self, order):
        if order.status in [bt.Order.Completed]:
            trade_id = self.order_ref_to_trade_id[order.ref]
            if trade_id in self.open_trades:
                self.log(f'Order executed: {order.ref}, Price: {order.executed.price}')
        elif order.status in [bt.Order.Canceled, bt.Order.Margin, bt.Order.Rejected]:
            self.log(f'Order Canceled/Margin/Rejected: {order.ref}')
            if order.ref in self.order_ref_to_trade_id:
                trade_id = self.order_ref_to_trade_id[order.ref]
                if trade_id in self.open_trades:
                    del self.open_trades[trade_id]
                del self.order_ref_to_trade_id[order.ref]

    def notify_trade(self, trade):
        if trade.isclosed:
            trade_ids_to_remove = []
            for order_ref, trade_id in self.order_ref_to_trade_id.items():
                if trade_id in self.open_trades:
                    del self.open_trades[trade_id]
                    trade_ids_to_remove.append(order_ref)
                    self.log(f'Trade: {trade_id}')
            for order_ref in trade_ids_to_remove:
                del self.order_ref_to_trade_id[order_ref]
            profit = trade.pnlcomm
            self.realized_daily_profit += profit
            self.realized_monthly_profit += profit
            if profit > 0:
                self.winning_streak += 1
                self.losing_streak = 0
            else:
                self.losing_streak += 1
                self.winning_streak = 0
            self.adjust_lot_size()

    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print(f'{dt.isoformat()}, {txt}')

    def next(self):
        self.manage_drawdown()
        self.check_daily_limits()
        self.update_equity_curve()
        self.execute_trading_signals()

    def execute_trading_signals(self):
        if self.get_total_buy_orders_count() == 0 and (self.check_buy_signal_trend() or self.check_buy_signal_counter()):
            tp_price_full = self.data.close[0] + (self.params.tp_in_pips * self.params._Point * 10)
            tp_price_partial = self.data.close[0] + (self.params.tp_in_pips * self.params.partial_profit_distance_pct / 100 * self.params._Point * 10)
            self.order = self.open_market_trade(self.current_lot_size, tp_price_partial, tp_price_full)
        if self.get_total_sell_orders_count() == 0 and (self.check_sell_signal_trend() or self.check_sell_signal_counter()):
            tp_price_full = self.data.close[0] - (self.params.tp_in_pips * self.params._Point * 10)
            tp_price_partial = self.data.close[0] - (self.params.tp_in_pips * self.params.partial_profit_distance_pct / 100 * self.params._Point * 10)
            self.order = self.open_market_trade(-self.current_lot_size, tp_price_partial, tp_price_full)

    def open_market_trade(self, lots, tp_price_partial, tp_price_full):
        order = self.buy(size=lots, exectype=bt.Order.Market) if lots > 0 else self.sell(size=-lots, exectype=bt.Order.Market)
        trade_id = f"{order.ref}_{self.data.datetime.date(0)}"
        self.order_ref_to_trade_id[order.ref] = trade_id
        self.open_trades[trade_id] = {
            'price': order.executed.price,
            'size': abs(lots),
            'type': 'buy' if lots > 0 else 'sell',
            'open_time': self.data.num2date(order.executed.dt),
            'tp_price_partial': tp_price_partial,
            'tp_price_full': tp_price_full
        }
        return order

    def check_buy_signal_trend(self):
        return (self.data.close[-2] < self.bbands.lines.top[-2] and
                self.data.close[-1] < self.bbands.lines.top[-1] and
                self.data.close[-1] > self.data.open[-1])

    def check_buy_signal_counter(self):
        return ((self.data.close[-2] > self.bbands.lines.bot[-2] and
                 self.data.close[-1] < self.bbands.lines.bot[-1]) or
                (self.data.close[-1] < self.bbands.lines.bot[-1] and
                 self.data.close[-1] > self.data.low[-1]))

    def check_sell_signal_trend(self):
        return (self.data.close[-2] > self.bbands.lines.bot[-2] and
                self.data.close[-1] > self.bbands.lines.bot[-1] and
                self.data.close[-1] < self.data.open[-1])

    def check_sell_signal_counter(self):
        return ((self.data.close[-2] < self.bbands.lines.top[-2] and
                 self.data.close[-1] > self.bbands.lines.top[-1]) or
                (self.data.close[-1] > self.bbands.lines.top[-1] and
                 self.data.close[-1] < self.data.high[-1]))

    def manage_drawdown(self):
        current_balance = self.broker.getvalue()
        if current_balance > self.highest_account_balance:
            self.highest_account_balance = current_balance
        drawdown = (self.highest_account_balance - current_balance) / self.highest_account_balance * 100
        if drawdown > self.params.close_on_drawdown_percent:
            self.log(f'Drawdown exceeded {self.params.close_on_drawdown_percent}%, closing all trades.')
            self.close_all_trades()

    def close_all_trades(self):
        for trade_id in list(self.open_trades.keys()):
            self.close(self.open_trades[trade_id]['size'])
            self.log('All positions closed due to drawdown.')

    def check_daily_limits(self):
        daily_loss_limit = self.starting_balance * (self.params.daily_sl / 100)
        current_balance = self.broker.getvalue()
        daily_loss = self.starting_balance - current_balance
        if daily_loss > daily_loss_limit:
            self.log(f'Daily loss limit exceeded: {daily_loss} > {daily_loss_limit}, all trades closed.')
            self.close_all_trades()
        elif self.realized_daily_profit > self.params.max_daily_profit:
            self.log('Daily profit target achieved, all trades closed.')
            self.close_all_trades()

    def adjust_lot_size(self):
        if self.winning_streak > 0:
            self.current_lot_size = max(self.params.initial_lot_size, self.current_lot_size / self.params.multiplier)
        elif self.losing_streak > 0:
            self.current_lot_size = min(self.params.max_lot_size, self.current_lot_size * self.params.multiplier)

    def update_equity_curve(self):
        current_balance = self.broker.getvalue()
        self.equity_curve.append(current_balance)
        peak = max(self.equity_curve, default=current_balance)
        drawdown = (peak - current_balance) / peak
        self.max_drawdown = max(self.max_drawdown, drawdown)

    def get_total_buy_orders_count(self):
        return sum(1 for trade_id, trade in self.open_trades.items() if trade['type'] == 'buy')

    def get_total_sell_orders_count(self):
        return sum(1 for trade_id, trade in self.open_trades.items() if trade['type'] == 'sell')

**GRID_SEARCH_OPTIMIZATION**

In [ ]:
# Load data
data = pd.read_csv('/content/1MIN_EURUSD_OHLC.csv', index_col='DateTime', parse_dates=True)
data_feed = bt.feeds.PandasData(dataname=data)

# Define the range of parameters to search
param_grid = {
    'tp_in_pips': [5, 7, 10],
    'multiplier': [1.05, 1.1],
    'max_lot_size': [7, 10],
    'bb_period': [20, 30],
    'bb_devfactor': [1, 2],
    'close_on_drawdown_percent': [3, 5, 8],
    'partial_profit_pct': [25, 50, 75],
    'partial_profit_distance_pct': [30, 50],
}

# Create a parameter grid
param_combinations = list(product(*param_grid.values()))
param_names = list(param_grid.keys())

best_params = None
best_profit = -np.inf
no_improvement_counter = 0
max_no_improvement = 10  # Stop after 10 iterations without improvement

for param_values in param_combinations:
    if no_improvement_counter >= max_no_improvement:
        break

    params = dict(zip(param_names, param_values))

    # Set up Cerebro
    cerebro = bt.Cerebro()
    cerebro.addstrategy(NapoleonEA, **params)
    cerebro.adddata(data_feed)
    cerebro.broker.set_cash(10000)
    cerebro.broker.setcommission(commission=0.0002)
    cerebro.addsizer(bt.sizers.FixedSize, stake=1)

    # Run the strategy
    results = cerebro.run()
    final_value = cerebro.broker.getvalue()

    # Check for best performance
    if final_value > best_profit:
        best_profit = final_value
        best_params = params
        no_improvement_counter = 0
    else:
        no_improvement_counter += 1

print('Best Parameters:', best_params)
print('Best Final Value:', best_profit)

# Save best parameters to JSON
results = {
    'best_params': best_params,
    'best_final_value': best_profit
}

with open('optimization_results.json', 'w') as f:
    json.dump(results, f, indent=4)